#### "PROMPT" Mask Example

In this example, we use a `maskPrompt` of _"sofa with pillows"_ to replace the sofa in the source image with a new sofa from the reference image.

_⚠️ **Warning:** Masks created with the “PROMPT” mask type will adhere tightly to the shape of the item you describe. This can be problematic in many scenarios because the product you are adding may not share the same silhouette or size as the item you are replacing. For this reason, the VTO API also provides an optional `maskShape` parameter that can be set to `“BOUNDING_BOX”`. We recommend using this setting (which is the default) in most cases when using the “PROMPT” mask type._

<div style="display: flex; gap: 2em; margin-right: 2em">
  <div style="text-align: center; flex: 1">
    <img src="../images/living_room-3.png">
    <strong><tt>sourceImage</tt></strong>
  </div>
  <div style="text-align: center; flex: 1">
    <img src="../images/small_room-sofa.png">
    <strong><tt>referenceImage</tt></strong>
  </div>
   <div style="text-align: center; flex: 1">
    <img src="../images/prompt_based_masking-output.png">
    <strong>Example output</strong>
  </div>
</div>


In [ ]:
import logging
import os
from datetime import datetime
from random import randint
from amazon_nova_canvas_utils import (
    generate_images,
    load_image_as_base64,
    base64_to_pil_image,
)
from IPython.display import Image

logging.basicConfig(format="[%(levelname)s] %(message)s", level=logging.INFO)

# Edit these values to experiment with your own images.
source_image_path = "../images/vto-images/vto_prompt_mask_source.jpg"
reference_image_path = "../images/vto-images/vto_prompt_mask_reference.jpg"

inference_params = {
    "taskType": "VIRTUAL_TRY_ON",
    "virtualTryOnParams": {
        "sourceImage": load_image_as_base64(source_image_path),
        "referenceImage": load_image_as_base64(reference_image_path),
        "maskType": "PROMPT",
        "promptBasedMask": {
            "maskPrompt":  "sofa with pillows",
            "maskShape": "BOUNDING_BOX",
        },
    },
    # The following is optional but provided here for you to experiment with.
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "quality": "standard",
        "cfgScale": 6.5,
        "seed": randint(0, 2147483646),
    },
}

output_folder = os.path.join("output", datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

try:
    response_body = generate_images(
        inference_params=inference_params,
        save_folder_path=output_folder,
        model_id="amazon.nova-canvas-v1:0",
        region_name="us-east-1",
    )

    # An error message may be returned, even if some images were generated.
    if "error" in response_body:
        logging.error(response_body["error"])

    if "images" in response_body:
        # Display all images.
        for image_base64 in response_body["images"]:
            image = base64_to_pil_image(image_base64)
            display(image)

except Exception as e:
    logging.error(e)

print(f"Done! Artifacts saved to {os.path.abspath(output_folder)}")